# EE 564 Project 2

## Tesla Model S Motor Design


[Tesla Model S](http://www.teslamotors.com/models)

[RWD 85 Model](http://en.wikipedia.org/wiki/Tesla_Model_S#Powertrain)

* Max. Power: 360 hp (270 kW)
* Max. Torque: 441 Nm
* Top Speed: 225 km/h 

[Specs1](http://teslatap.com/undocumented/) and [Specs2](http://my.teslamotors.com/it_IT/forum/forums/tesla-model-s-motor-output)

**Useful Links:**

* [How does Tesla make their motors in the model S so small?](http://www.reddit.com/r/AskEngineers/comments/2mv2h5/how_does_tesla_make_their_motors_in_the_model_s/)

* [Tesla Motor Technology](http://my.teslamotors.com/roadster/technology/motor)

* [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf)

## Design Specifications

From [Specs1](http://teslatap.com/undocumented/): "Motor Inverter – This converts the DC battery voltage to AC, powering the motor. It delivers up to 1000 amps (from 2010 10K annual report)."

In [292]:
import math
Pn=270e3 # Max. power (W)
Iline=1000 # Max. current (A)
Vll=Pn/(math.sqrt(3)*Iline) # Max. L-L voltage (V)
Vln=Vll/math.sqrt(3) # Max. L-N voltage (V)
print Vll
print Vln

155.884572681
90.0


In [293]:
Vll=200 # Design motor with 200 V rating
Vln=200/math.sqrt(3)
f1=60 # Supply frequency (Hz)
m=3 # Number of phases, Star-connected
nn=0.96 # Target efficiency
cosphin=0.83 # Target power factor

From [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf) Slide 38;

In [294]:
p1=2 # Number of pole pairs

## Main Dimensions of Stator Core

Reference: I. Boldea and S.A. Nasar, "The Induction Machine Handbook", CRC Press LLC, 2002.

Equation, figure and table numbers are given as comments next to the equation.

In [295]:
KE=0.98-0.005*p1 # Eq. 15.1
Sgap=KE*Pn/(nn*cosphin) # Apparent power (VA), Eq. 15.2
print KE
print Sgap

0.97
328689.759036


In [296]:
C0=250e3 # Esson's constant (J/m^3), Fig. 14.14
lambda1=1.5 # Stack aspect ratio, Table 15.1
Dis=math.pow((2*p1*p1*Sgap)/(math.pi*lambda1*f1*C0),1.0/3.0) # Stator bore diameter (m), Eq. 15.1
print Dis

0.333821719418


In [297]:
tau=math.pi*Dis/(2*p1) # Pole pitch (m), Eq. 15.2
L=lambda1*tau # Stack length (m), Eq. 15.2
print tau
print L

0.262182965333
0.393274447999


From [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf) Slide 40;

In [298]:
q=4 # Number of stator slots per pole per phase
taus=tau/(m*q) # Slot pitch (m), Eq. 15.3
print taus

0.0218485804444


In [299]:
KD=0.62 # Dis/Dout, Table 15.2
Dout=Dis/KD # Stator outer diameter (m), Eq. 15.4
print Dout

0.538422128093


In [300]:
g=(0.1+0.012*math.pow(Pn,1.0/3.0))*1e-3 # Air gap length (m), Eq. 15.5
print g

0.000875596488411


## Stator Winding

In [301]:
Ns=2*p1*q*m # Number of stator slots, Eq. 15.6
print Ns

48


Two-layer winding with chorded coils, y/$\tau$=10/12 is chosen as 10/12=0.83 is close to 0.8, which would reduce the first (5$^{th}$ order) stator mmf space harmonic.

In [302]:
alphaec=2*math.pi*p1/Ns # Electrical angle between emfs in neighboring slots (rad), Eq. 15.7
print alphaec

0.261799387799


The distribution of phases in slots will be given by MotorAnalysis software later.

In [303]:
Kq1=math.sin(q*alphaec/2)/(q*math.sin(alphaec/2)) # Distribution factor, Eq. 15.8
print Kq1

0.957662196943


In [304]:
Ky1=math.sin(math.pi/2*10/12) # Pitch factor, Eq. 15.9
print Ky1

0.965925826289


In [305]:
Kw1=Kq1*Ky1 # Winding factor
print Kw1

0.925030648888


In [306]:
Bg=0.65 # Recommended air gap flux density (T), Eq. 15.11
Kf=1.085
alphai=0.729 # Pole spanning coefficient, Fig. 14.13
phi=alphai*tau*L*Bg # Pole flux (Wb), Eq. 15.10
print phi


0.0488586076188


In [307]:
W1=KE*Vln/(4*Kf*Kw1*f1*phi) # Number of turns per phase, Eq. 15.12
print W1

9.51706233954


In [308]:
a1=1 # Number of current paths in parallel
ns=a1*W1/(p1*q) # Number of conductors per slot, Eq. 15.13
print ns

1.18963279244


This should be an even number since there are two distinct coils per slot in a double layer winding.

In [309]:
ns=2
W1=p1*q*ns
print W1

16


In [310]:
Bg=Bg*9.517/16 # Actual air gap flux density (T), Eq. 15.15
print Bg

0.386628125


At this point Bg was increased from 0.386 T to 0.45 T in order to satisfy $F_{mtr} \approx F_{mts}$.

In [311]:
Bg=0.45

In [312]:
Iln=Pn/(3*Vln*cosphin*nn) # Rated current (A), Eq. 15.16
print Iln

978.191344636


In [313]:
Jcos=6 # Current density (A/mm^2), Eq. 15.17
Aco=Iln/(Jcos*a1) # Magnetic wire cross section (mm^2), Eq. 15.18
print Aco

163.031890773


Circular cross section area is to be used. At 60 Hz skin depth is ~ 9 mm # VERIFY

In [314]:
dco=math.sqrt(4*Aco/math.pi) # Wire gauge diameter (mm), Eq. 15.19
print dco

14.407590027


We can use few conductors in parallel, $a_p$=8.

In [315]:
ap=8
dco=math.sqrt(4*Aco/(math.pi*ap)) # Wire gauge diameter (mm), Eq. 15.19
print dco

5.09385230431


From [AWG](http://en.wikipedia.org/wiki/American_wire_gauge); AWG4 wire is selected.

In [316]:
A1=C0/(Kf*alphai*Kw1*3.14*Bg*2*p1)
print A1

60453.9199497


TO BE DELETED ^

## Stator Slot Sizing

In [317]:
Kfill=0.44 # Fill factor for round wire
Asu=math.pi*math.pow(dco,2)*ap*ns/(4*Kfill) # Useful slot area (mm^2), Eq. 15.21
print Asu

741.054048967


Stator slot geometry: Fig. 15.4 (a)

In [318]:
Kfe=0.96 # For 0.5 mm thick lamination
Bts=1.55 # Tooth flux density (T)
bts=Bg*taus*L/(Bts*L*Kfe) # Tooth width (m), Eq. 15.22
print bts

0.00660743360214


In [319]:
bos=2.2e-3 # Fig. 15.5
hos=1e-3 # Fig. 15.5
hw=1.5e-3 # Fig. 15.5
bs1=math.pi*(Dis+2*hos+2*hw)/Ns-bts # Slot lower width (m), Eq. 15.23
print bs1

0.015568396077


In [320]:
bs2=math.sqrt(4*Asu*1e-6*math.tan(math.pi/Ns)+math.pow(bs1,2)) # Upper slot width (m), Eq. 15.27
print bs2

0.0208964099888


In [321]:
hs=2*Asu*1e-6/(bs1+bs2) # Slot useful height (m), Eq. 15.28
print hs

0.0406448918241


In [322]:
Fmg=1.2*g*Bg/(4*math.pi*1e-7) # Air gap mmf (AT), Eq. 15.29
print Fmg

376.259875068


In [323]:
Hts=1760 # Magnetic field intensity for Bts=1.55 T (A/m)
Fmts=Hts*(hs+hos+hw) # Stator tooth mmf (AT), Eq. 15.30
print Fmts

75.9350096105


In [324]:
Kst=1.4-1 # Teeth saturation factor, Eq. 15.29
Fmtr=Kst*Fmg-Fmts # Rotor tooth mmf (AT), Eq. 15.31
print Fmtr

74.5689404169


At this point Bg was increased from 0.386 T to 0.45 T in order to satisfy $F_{mtr} \approx F_{mts}$.

In [325]:
hcs=(Dout-(Dis+2*(hos+hw+hs)))/2 # Stator back iron height (m), Eq. 15.32
print hcs

0.0591553125135


In [326]:
Bcs=phi/(2*L*hcs) # Back core flux density (T), Eq. 15.33
print Bcs

1.05007811508


which is a reasonable value.

## Rotor Slots

From [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf) Slide 40;

In [327]:
Nr=68 # Number of rotor slots
K1=0.8*cosphin+0.2 # Rotor to stator mmf ratio, Eq. 15.35
Ib=K1*(2*m*W1*Kw1)/Nr*Iln # Rotor bar current (A), Eq. 15.34
print Ib

1103.71260108


Rotor slot geometry: Fig. 15.6 (c)

In [328]:
Jb=6 # Current density in the rotor bar (A/mm^2)
Ab=Ib/Jb*1e-6 # Rotor slot area (m^2), Eq. 15.36
print Ab

0.00018395210018


In [329]:
Icr=Ib/(2*math.sin(math.pi*p1/Nr)) # End ring current (A), Eq. 15.37
print Icr

5980.99179123


In [330]:
Jcr=0.8*Jb # Current density in the end ring (A/mm^2)
Acr=Icr/Jcr*1e-6 # End ring cross section (m^2), Eq. 15.38
print Acr

0.00124603995651


In [331]:
hor=0.5e-3 # Fig. 15.7
bor=1.5e-3 # Fig. 15.7
taur=math.pi*(Dis-2*g)/Nr # Rotor slot pitch (m), Eq. 15.39
print taur

0.0153416224462


In [332]:
Btr=1.60 # Rotor tooth flux density
btr=Bg/(Kfe*Btr)*taur # Rotor tooth width (m), Eq. 15.40
print btr

0.00449461595103


In [333]:
Dre=Dis-g # Rotor diameter (m)
print Dre
d1=(math.pi*(Dre-2*hor)-Nr*btr)/(math.pi+Nr) # Rotor slot upper diameter (m), Eq. 15.42
print d1

0.332946122929
0.0103625121257
